# Purpose 

### 商品・日付・店舗区分から販売数を予測
* * *

# Agenda

1. 商品情報を説明変数とする
1. 日付×店舗区分から、店舗毎の販売数を当てる時系列分析


## import

In [58]:
% matplotlib inline
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
import bokeh.plotting as bplt
from IPython.display import display, HTML
bplt.output_notebook()

Loading BokehJS ...

In [52]:
datanames  = ['product', 'sales', 'store', 'test', 'train']
sex = ['m', 'w']
age = ['00_19', '20_49', '50_']
segments = ['m00_19', 'm20_49', 'm50_', 'w00_19', 'w20_49', 'w50_']
pids = ['p000', 'p001', 'p002', 'p003', 'p004', 'p005', 'p006', 'p007',
       'p008', 'p009', 'p010', 'p011', 'p012', 'p013', 'p014', 'p015',
       'p016', 'p017', 'p018', 'p019', 'p020', 'p021', 'p022', 'p023',
       'p024', 'p025', 'p026', 'p027', 'p028', 'p029', 'p030', 'p031',
       'p032', 'p033', 'p034', 'p035', 'p036', 'p037', 'p038', 'p039',
       'p040', 'p041', 'p042', 'p043', 'p044', 'p045', 'p046', 'p047',
       'p048', 'p049', 'p050', 'p051', 'p052', 'p053', 'p054', 'p055',
       'p056', 'p057', 'p058', 'p059', 'p060', 'p061', 'p062', 'p063',
       'p064', 'p065', 'p066', 'p067', 'p068', 'p069', 'p070', 'p071',
       'p072']
train_dates = [201506, 201507, 201508, 201509, 201510, 201511, 201512, 201601, 201602, 201603, 201604, 201605]
store_catecol = ['area', 'location']
size_dict = {'size_g': 'g', 'size_stick':'本', 'size_piece':'個', 'size_sheet':'枚'}
# 本 -> stick for index[50] , 個 -> stick for index[14, 15, 25, 37] , 枚 -> stick for index[27, 28] 
product_catecol = ['category', 'package_type', 'genre', 'manufacturer']
product_apeals = ['cal', '食物繊維', '乳酸菌', 'オリーブオイル', '砂糖', '糖類', '糖質', '食塩']
allergys= ['allergy_egg', 'allergy_wheat', 'allergy_milk', 'allergy_peanut', 'allergy_shrimp']

In [39]:
df_dict_origin = {datanames[n] : pd.read_csv('../data/{}.tsv'.format(name), delimiter='\t') for n, name in enumerate(datanames)}
df_dict = {name : df_dict_origin[name].copy() for name in df_dict_origin}

## Pre-paration

### store's categorical variables -> dummy

* store_catecol = ['area', 'location']のダミー変数を生成
* store_catecol = ['area', 'location']をドロップ

In [41]:
for df_n in ['train', 'test']:
    for col in store_catecol:
        dummy = pd.get_dummies(df_dict[df_n][col])
        df_dict[df_n] = pd.concat((df_dict[df_n], dummy), axis=1)
    df_dict[df_n] = df_dict[df_n].drop(store_catecol, axis =1)

### 商品のサイズ種類

* [g, 枚, 個, 本]にカテゴライズ
* 対応するものにfloatで保管
* 違うカラムには０を保存

In [42]:
df_n = 'product'
col_n = 'size'
entity = 'g'

df_dict[df_n][df_dict[df_n][col_n].str.contains(entity).apply(lambda x: False if x else True)][col_n]

14     4個
15     4個
25     7個
27     6枚
28     5枚
37     7個
50    10本
Name: size, dtype: object

In [43]:
df_n = 'product'
col_n = 'size'
decode_n = 'utf-8'
dtype_d = 'float64'

for size_n in size_dict:
    df_dict[df_n][size_n] = df_dict[df_n][col_n].apply(lambda x: x.decode(decode_n)[:-1]).astype(dtype_d)
    df_dict[df_n].loc[df_dict[df_n][col_n].str.contains(size_dict[size_n]).apply(lambda x: False if x else True), size_n] = 0
df_dict[df_n].drop(col_n, axis=1)

,pid,category,pname,price,package_type,abstract,appeal_point,allergy_egg,allergy_wheat,allergy_milk,...,nutrition_saturatedFat(g),nutrition_transFat(g),nutrition_cholesterol(mg),nutrition_lactose(g),nutrition_pantothenicAcid(mg),materials,size_g,size_stick,size_sheet,size_piece
0,p000,ナチュラルローソン菓子,うましおポテト（ノンフライ）,148,カップ,ザク切りポテトを、カリッと焼き上げました。,126kcal,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,馬鈴薯（遺伝子組換えでない）、植物油、砂糖、食塩、デキストリン、食物繊維、ブドウ糖、たんぱく...,30.0,0,0,0
1,p001,ナチュラルローソン菓子,くるみとココナッツのキャラメリゼ素焼きアーモンド入り,198,袋,くるみとココナッツに、キャラメルのやさしい甘さが広がります。,食物繊維3.8g,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,クルミ、ココナッツ、アーモンド、グラニュー糖、砂糖、ショ糖、でん粉、ぶどう糖、デキストリン、...,40.0,0,0,0
2,p002,ナチュラルローソン菓子,こんにゃくチップス コンソメ風味,148,袋,さっくり軽い、こんにゃく素材のスナックです。,63kcal,NaN,1,NaN,...,0.2,0,0,NaN,NaN,こんにゃく粉、植物油、砂糖、コンソメシーズニング（砂糖、ローストチキンパウダー、食塩、オニオ...,15.0,0,0,0
3,p003,ナチュラルローソン菓子,こんにゃくチップスのりしお風味,148,袋,さっくり軽い、こんにゃく素材のスナックです。,61kcal,NaN,NaN,NaN,...,0.2,0,0,NaN,NaN,こんにゃく粉、植物油、砂糖、食塩、アオサ、和風だし（食塩、ぶどう糖、鰹節粉末、昆布エキス、た...,15.0,0,0,0
4,p004,ナチュラルローソン菓子,エンゼルパイミニさわやか苺味,148,袋,もちもちマシュマロと、しっとりビスケット。２つの食感を、ひと口サイズで。,乳酸菌200億個配合,1,1,1,...,NaN,NaN,NaN,NaN,NaN,小麦粉、準チョコレート（乳成分を含む）、水あめ、砂糖、ショートニング、麦芽糖、鶏卵、オリゴ糖...,45.0,0,0,0
5,p005,ナチュラルローソン菓子,オリーブオイルで揚げたポテトチップス,148,袋,オリーブオイルとイタリア産天日塩を使用。食べごたえのある食感に仕上げました。,オリーブオイル使用,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,じゃがいも（遺伝子組換えでない）、オリーブオイル、食塩（イタリア産天日塩100%使用）,40.0,0,0,0
6,p006,ナチュラルローソン菓子,キヌアと８種のグラノーラチョコレート,148,袋,キアヌや発芽玄米などの穀物をブレンド。サクッと食感に仕上げました。,130kcal,NaN,1,1,...,NaN,NaN,NaN,NaN,0.3,砂糖、ココアバター、コーンフレーク（コーングリッツ、砂糖、麦芽エキス、食塩、ぶどう糖果糖液糖...,26.0,0,0,0
7,p007,ナチュラルローソン菓子,シナモン香るアーモンド,198,袋,カリッと食感のアーモンドに、シナモンが華やかに香ります。,食物繊維4.4g,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,アーモンド、砂糖、小麦粉、でん粉、還元水あめ、シナモンパウダー、でん粉分解物、食塩、着色料（...,38.0,0,0,0
8,p008,ナチュラルローソン菓子,ゼロ ノンシュガーチョコレート,148,袋,スイートとビター、2層のチョコがなめらかに溶け合います。,砂糖ゼロ 糖類ゼロ,NaN,NaN,1,...,NaN,NaN,NaN,0,NaN,カカオマス、マルチトール、植物油脂、乳等を主要原料とする食品（食物繊維、バター、分離乳タンパ...,26.0,0,0,0
9,p009,ナチュラルローソン菓子,ゼロ ノンシュガービスケット,148,袋,全粒粉香るやさしい甘さのビスケットです。,砂糖ゼロ・糖類ゼロ,1,1,1,...,NaN,NaN,NaN,0,NaN,小麦粉、マルチトール、ショートニング、小麦全粒粉、乳等を主要原料とする食品、全卵、食物繊維、...,21.0,0,0,0


### 商品のカテゴリカル変数 -> ダミー変数

* category :  [おやつごろシリーズ菓子, ナチュラルローソン菓子]
* package_type : [カップ, 箱, 袋]
* genre : (total 24)[u'クッキー', u'クラッカー', u'スナック菓子', u'チョコレート', u'チョコレート菓子', u'ドーナツ', u'パイ', u'パイ加工品', u'ビスケット', u'ポテトチップス', u'マシュマロ', u'ラスク', u'半生菓子', u'有機干し芋', u'油菓子', u'準チョコレート', u'焼菓子', u'種実加工品', u'米菓', u'米菓子', u'菓子', u'野菜チップス', u'魚介乾製品', u'魚介加工品']
* manufacturer : (total 40) [u'ぼんち株式会社', u'イケダヤ製菓株式会社', u'イトウ製菓株式会社', u'カバヤ食品株式会社', u'カルビー株式会社',u'ジャパンフリトレー株式会社', u'ダイシンフーズ株式会社', u'ニッポー株式会社', u'ハース株式会社', u'ホンダ製菓株式会社',u'リスカ株式会社', u'ローヤル製菓株式会社', u'久保田製菓有限会社', u'宝製菓株式会社', u'旺旺ジャパン株式会社',u'東京カリント株式会社', u'株式会社おやつカンパニー', u'株式会社エイワ', u'株式会社オリーヴ', u'株式会社カレイナ',u'株式会社ギンビス', u'株式会社ニッコー', u'株式会社ミツヤ', u'株式会社リボン', u'株式会社ロッテ',u'株式会社不二家', u'株式会社北食', u'株式会社合食', u'株式会社壮関', u'株式会社天乃屋', u'株式会社末広製菓',u'株式会社村田製菓', u'株式会社東ハト', u'株式会社栗山米菓', u'株式会社正栄デリシィ', u'株式会社湖池屋',u'株式会社道南冷蔵', u'森永製菓株式会社', u'江崎グリコ株式会社', u'阿部幸製菓株式会社']

In [44]:
df_n = 'product'
for col in product_catecol:
    dummy = pd.get_dummies(df_dict[df_n][col])
    df_dict[df_n] = pd.concat((df_dict[df_n], dummy), axis=1)
df_dict[df_n] = df_dict[df_n].drop(product_catecol, axis =1)

### 商品のアピールポイント -> ダミー変数

* [cal, 食物繊維, 乳酸菌, オリーブオイル, 砂糖, 糖類, 糖質, 食塩]
* アピールポイントに上記のワードを含む商品にフラグをたてる

In [45]:
df_dict[df_n].appeal_point[df_dict[df_n].appeal_point.apply(pd.notnull)]

0          126kcal
1         食物繊維3.8g
2           63kcal
3           61kcal
4       乳酸菌200億個配合
5        オリーブオイル使用
6          130kcal
7         食物繊維4.4g
8        砂糖ゼロ　糖類ゼロ
9        砂糖ゼロ・糖類ゼロ
10        食物繊維2.1g
11    乳酸菌2億個（5枚当り）
12    乳酸菌2億個（5枚当り）
13        食物繊維2.1g
14          糖質7.5g
15          糖質7.5g
16         174kcal
17         砂糖食塩不使用
18        食物繊維3.6g
19           砂糖不使用
20          乳酸菌2億個
21         168kcal
22        食物繊維2.5g
23        食物繊維5.2ｇ
24         砂糖食塩不使用
25          糖質9.3g
Name: appeal_point, dtype: object

In [46]:
df_n = 'product'
col_n = 'appeal_point'

df_dict[df_n][col_n] = df_dict[df_n][col_n].fillna('tmp')
for word in product_apeals:
    df_dict[df_n][word] = df_dict[df_n].appeal_point.apply(lambda x: 1 if word in x else 0)
df_dict[df_n] = df_dict[df_n].drop(col_n, axis =1)


### アレルギー情報の0補間

In [53]:
df_n = 'product'

for col_n in allergys:
    df_dict[df_n][col_n] = df_dict[df_n][col_n].fillna(0)

### 説明文の有無フラグ

In [69]:
df_n = 'product'
col_n = 'description'

df_dict[df_n][col_n] =  df_dict[df_n][col_n].apply(lambda x: 0 if pd.isnull(x) else 1)

### 原材料のダミー変数

In [88]:
pd.isnull(df_dict['product'].materials).value_counts()

False    73
Name: materials, dtype: int64

In [94]:
df_dict['product'].materials[0:2]

0    馬鈴薯（遺伝子組換えでない）、植物油、砂糖、食塩、デキストリン、食物繊維、ブドウ糖、たんぱく...
1    クルミ、ココナッツ、アーモンド、グラニュー糖、砂糖、ショ糖、でん粉、ぶどう糖、デキストリン、...
Name: materials, dtype: object

### 栄養成分量

In [30]:
for col in df_dict[datanames[0]].columns:
    if df_dict[datanames[0]][col].dtype != 'float64':
        continue
    if df_dict[datanames[0]][df_dict[datanames[0]][col] == 0].size != 0:
        print col

nutrition_fat(g)
nutrition_sugars(g)
nutrition_sucrose(g)
nutrition_transFat(g)
nutrition_cholesterol(mg)
nutrition_lactose(g)
size_g
size_stick
size_sheet
size_piece
おやつごろシリーズ菓子
ナチュラルローソン菓子
カップ
箱
袋
クッキー
クラッカー
スナック菓子
チョコレート
チョコレート菓子
ドーナツ
パイ
パイ加工品
ビスケット
ポテトチップス
マシュマロ
ラスク
半生菓子
有機干し芋
油菓子
準チョコレート
焼菓子
種実加工品
米菓
米菓子
菓子
野菜チップス
魚介乾製品
魚介加工品
ぼんち株式会社
イケダヤ製菓株式会社
イトウ製菓株式会社
カバヤ食品株式会社
カルビー株式会社
ジャパンフリトレー株式会社
ダイシンフーズ株式会社
ニッポー株式会社
ハース株式会社
ホンダ製菓株式会社
リスカ株式会社
ローヤル製菓株式会社
久保田製菓有限会社
宝製菓株式会社
旺旺ジャパン株式会社
東京カリント株式会社
株式会社おやつカンパニー
株式会社エイワ
株式会社オリーヴ
株式会社カレイナ
株式会社ギンビス
株式会社ニッコー
株式会社ミツヤ
株式会社リボン
株式会社ロッテ
株式会社不二家
株式会社北食
株式会社合食
株式会社壮関
株式会社天乃屋
株式会社末広製菓
株式会社村田製菓
株式会社東ハト
株式会社栗山米菓
株式会社正栄デリシィ
株式会社湖池屋
株式会社道南冷蔵
森永製菓株式会社
江崎グリコ株式会社
阿部幸製菓株式会社


In [31]:
df_dict[datanames[0]][df_dict[datanames[0]]['nutrition_fat(g)'] == 2.8].size

114

In [32]:
df_dict[datanames[0]]['nutrition_fat(g)']

0      2.8
1     22.0
2      2.5
3      2.1
4      7.5
5      9.3
6      6.6
7     10.4
8     11.3
9      3.8
10    10.2
11     5.1
12     5.2
13     8.9
14    10.9
15    10.9
16     5.5
17    12.7
18    11.0
19    12.6
20     0.0
21     2.2
22    15.0
23     0.4
24     8.3
25     9.2
26    11.0
27    18.9
28    11.4
29     1.0
      ... 
43    14.7
44    16.2
45    28.0
46    34.8
47    22.6
48    13.4
49    12.1
50    18.6
51    22.1
52    13.9
53    27.4
54    27.3
55    20.0
56    22.8
57     9.5
58    26.1
59    15.8
60    18.3
61     0.6
62    32.1
63    23.4
64    21.1
65    10.5
66    20.5
67    14.4
68    21.7
69    18.0
70    22.0
71     3.0
72    20.3
Name: nutrition_fat(g), dtype: float64

In [91]:
searchs

[Empty DataFrame
 Columns: [pid, category, pname, price, package_type, abstract, appeal_point, allergy_egg, allergy_wheat, allergy_milk, allergy_peanut, allergy_shrimp, description, genre, size, manufacturer, nutrition_calories(kcal), nutrition_protein(g), nutrition_fat(g), nutrition_carbohydrate(g), nutrition_sodium(mg), nutrition_salt(g), nutrition_sugars(g), nutrition_dietaryFiber(g), nutrition_sucrose(g), nutrition_collagen(mg), nutrition_calcium(mg), nutrition_vitaminD(ug), nutrition_vitaminB1(mg), nutrition_vitaminB2(mg), nutrition_vitaminE(mg), nutrition_iron(mg), nutrition_saturatedFat(g), nutrition_transFat(g), nutrition_cholesterol(mg), nutrition_lactose(g), nutrition_pantothenicAcid(mg), materials]
 Index: []
 
 [0 rows x 38 columns], Empty DataFrame
 Columns: [pid, category, pname, price, package_type, abstract, appeal_point, allergy_egg, allergy_wheat, allergy_milk, allergy_peanut, allergy_shrimp, description, genre, size, manufacturer, nutrition_calories(kcal), nutrition_

In [64]:
[df_dict[datanames[0]].columns[n] for n, search in enumerate(searchs)  if search.size != 0 ]

['appeal_point',
 'allergy_wheat',
 'allergy_milk',
 'nutrition_fat(g)',
 'nutrition_carbohydrate(g)',
 'nutrition_sodium(mg)']